# **RL In-Class Exercises Tutorial 1**
This notebook contains the in-class exercises from tutorial 1 of Deep Reinforcement Learning in AML.

# SETUP - Run this cell first

In [1]:
import torch
import torch.nn as nn
from torch.distributions.normal import Normal
import numpy as np
from typing import Sequence

def mlp(sizes: Sequence[int], activation=nn.ReLU, output_activation=nn.Identity) -> nn.Sequential:
  """
  Create a simple feedforward neural network.

  Args:
      sizes: List of layer sizes [input_dim, hidden1, hidden2, ..., output_dim]
      activation: Activation function for hidden layers
      output_activation: Activation function for output layer

  Returns:
      nn.Sequential: The neural network
  """
  layers = []
  for j in range(len(sizes)-1):
    act = activation if j < len(sizes)-2 else output_activation
    layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
  return nn.Sequential(*layers)

print("✓ Setup complete! Ready to start exercises.")


✓ Setup complete! Ready to start exercises.


# EXERCISE 1: Implement discounted_return

## TODO: Implement discounted_return

Weight current reward t more than future ones because the more t enlarges, le smaller lambda will be given that it is between 0 and 1 and as t increases, so does it power.

Implementation wants this :
j = 1
R_t = r_t + gamma^j * r_t


In [43]:
def discounted_return(arr: Sequence[float], gamma: float = 0.99) -> list[float]:
    """
    Compute the discounted return (rewards-to-go) for a single episode.

    This computes the sum of future discounted rewards from each timestep.
    For example, at timestep t: R_t = r_t + gamma*r_{t+1} + gamma^2*r_{t+2} + ...

    Args:
        arr: Sequence of rewards for one episode, e.g. [r_0, r_1, r_2, ..., r_T]
        gamma: Discount factor in (0, 1]

    Returns:
        List of discounted returns, same length as arr
    """
    R_t = []
    for i in range(len(arr)):
        sum = 0
        power = 1
        for j in range(i, len(arr)):
            if j == i :
                sum += arr[j]
            else:
                sum += gamma**power *arr[j]
                power += 1
        R_t.append(sum)
    return R_t

## Test Functions (Feel free to collapse this cell)

In [44]:
def test_discounted_return(discounted_return_fn):
    """
    Test the discounted_return implementation.
    """
    results = []

    # Test 1: Output is a list
    try:
        rewards = [1.0, 2.0, 3.0]
        returns = discounted_return_fn(rewards, gamma=0.99)

        if isinstance(returns, list):
            print("✓ Test 1: Output is a list")
            results.append(True)
        else:
            print(f"✗ Test 1: Output should be a list")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 1: Error - {type(e).__name__}")
        results.append(False)

    # Test 2: Output length matches input length
    try:
        for length in [1, 5, 10, 100]:
            rewards = [1.0] * length
            returns = discounted_return_fn(rewards, gamma=0.99)

            if len(returns) != length:
                print(f"✗ Test 2: Length mismatch for input length {length}")
                results.append(False)
                break
        else:
            print("✓ Test 2: Output length matches input length")
            results.append(True)
    except Exception as e:
        print(f"✗ Test 2: Error - {type(e).__name__}")
        results.append(False)

    # Test 3: Output elements are floats
    try:
        rewards = [1.0, 2.0, 3.0]
        returns = discounted_return_fn(rewards, gamma=0.99)

        if all(isinstance(x, float) for x in returns):
            print("✓ Test 3: Output elements are floats")
            results.append(True)
        else:
            print("✗ Test 3: Output elements should be floats")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 3: Error - {type(e).__name__}")
        results.append(False)

    # Test 4: Last element equals last reward
    try:
        rewards = [1.0, 2.0, 3.0, 5.0]
        returns = discounted_return_fn(rewards, gamma=0.99)

        if abs(returns[-1] - rewards[-1]) < 1e-6:
            print("✓ Test 4: Last element equals last reward")
            results.append(True)
        else:
            print("✗ Test 4: Last element should equal last reward")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 4: Error - {type(e).__name__}")
        results.append(False)

    # Test 5: Simple numerical correctness
    try:
        # With gamma=0.9 and rewards [1, 1, 1]:
        # returns = [1 + 0.9*1 + 0.81*1, 1 + 0.9*1, 1] = [2.71, 1.9, 1.0]
        rewards = [1.0, 1.0, 1.0]
        returns = discounted_return_fn(rewards, gamma=0.9)
        expected = [2.71, 1.9, 1.0]

        if all(abs(returns[i] - expected[i]) < 0.01 for i in range(len(returns))):
            print("✓ Test 5: Numerical correctness")
            results.append(True)
        else:
            print("✗ Test 5: Numerical values incorrect")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 5: Error - {type(e).__name__}")
        results.append(False)

    # Test 6: Different gamma values work
    try:
        rewards = [1.0, 2.0, 3.0]
        returns1 = discounted_return_fn(rewards, gamma=0.9)
        returns2 = discounted_return_fn(rewards, gamma=0.5)

        # With different gammas, returns should be different
        if returns1 != returns2:
            print("✓ Test 6: Different gamma values produce different results")
            results.append(True)
        else:
            print("✗ Test 6: Different gammas should produce different results")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 6: Error - {type(e).__name__}")
        results.append(False)

    return sum(results), len(results)

## Run Tests

In [45]:
# Run the tests for discounted_return
passed, total = test_discounted_return(discounted_return)
print(f"\n{'='*50}")
print(f"Results: {passed}/{total} tests passed")
if passed == total:
  print("🎉 All tests passed! You're ready to move on.")
else:
  print("⚠️  Some tests failed. Review your implementation.")

✓ Test 1: Output is a list
✓ Test 2: Output length matches input length
✓ Test 3: Output elements are floats
✓ Test 4: Last element equals last reward
✓ Test 5: Numerical correctness
✓ Test 6: Different gamma values produce different results

Results: 6/6 tests passed
🎉 All tests passed! You're ready to move on.


# EXERCISE 2: Implement compute_loss

## Helper function for Exercise 2

In [73]:
def get_policy(obs: torch.Tensor) -> Normal:
  """
  Simplified policy that returns a random Normal distribution.
  This is a placeholder - you'll implement the real version in a later exercise.

  Args:
      obs: Observation tensor of shape (batch_size, obs_dim) or (obs_dim,)

  Returns:
      Normal distribution for actions
  """
  obs = obs.unsqueeze(0) if obs.dim() == 1 else obs
  batch_size = obs.shape[0]
  n_acts = 2  # Hardcoded for this exercise

  # Return random Normal distributions
  mean = torch.randn(batch_size, n_acts)+ obs.sum() * 0.0
  std = torch.ones(batch_size, n_acts)
  return Normal(mean, std)

## TODO: Implement compute_loss

In [74]:
def compute_loss(obs: torch.Tensor, act: torch.Tensor, weights: torch.Tensor) -> torch.Tensor:
    """
    Compute the REINFORCE loss for a given batch.

    The loss is the negative weighted log-probability of actions.

    Args:
        obs: Observations tensor of shape (batch_size, obs_dim)
        act: Actions tensor of shape (batch_size, n_acts)
        weights: Weights (returns-to-go) tensor of shape (batch_size,)

    Returns:
        loss: Scalar tensor representing the negative weighted log-probability
    """
    # NOTE: here we return the normal distribution given by the heper function
    dist = get_policy(obs)
    # NOTE: Moreover we need the log probability 
    #       Sum over the batch dimension (number of episode we run the update)
    log_pb = dist.log_prob(act).sum(dim=-1)
    # NOTE: Finally take the mean
    return -(log_pb * weights).mean()

## Test Functions (Feel free to collapse this cell)

In [75]:
def test_compute_loss(compute_loss_fn):
    """
    Test the compute_loss implementation.
    """
    results = []

    # Test 1: Output is a scalar
    try:
        obs = torch.randn(10, 8)
        act = torch.randn(10, 2)
        weights = torch.randn(10)
        loss = compute_loss_fn(obs, act, weights)

        if loss.shape == torch.Size([]):
            print("✓ Test 1: Output is scalar")
            results.append(True)
        else:
            print(f"✗ Test 1: Output should be scalar, got shape {loss.shape}")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 1: Error - {type(e).__name__}")
        results.append(False)

    # Test 2: Output dtype is correct
    try:
        obs = torch.randn(10, 8)
        act = torch.randn(10, 2)
        weights = torch.randn(10)
        loss = compute_loss_fn(obs, act, weights)

        if loss.dtype == torch.float32:
            print("✓ Test 2: Output dtype is correct")
            results.append(True)
        else:
            print(f"✗ Test 2: Output dtype should be float32, got {loss.dtype}")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 2: Error - {type(e).__name__}")
        results.append(False)

    # Test 3: Works with different batch sizes
    try:
        torch.manual_seed(42)
        for batch_size in [1, 10, 100]:
            obs = torch.randn(batch_size, 8)
            act = torch.randn(batch_size, 2)
            weights = torch.randn(batch_size)
            loss = compute_loss_fn(obs, act, weights)

            if loss.shape != torch.Size([]):
                print(f"✗ Test 3: Failed for batch_size={batch_size}")
                results.append(False)
                break
        else:
            print("✓ Test 3: Works with different batch sizes")
            results.append(True)
    except Exception as e:
        print(f"✗ Test 3: Error - {type(e).__name__}")
        results.append(False)

    # Test 4: Different weights produce different losses
    try:
        torch.manual_seed(42)
        obs = torch.randn(10, 8)
        act = torch.randn(10, 2)
        weights1 = torch.ones(10)
        weights2 = torch.ones(10) * 2

        loss1 = compute_loss_fn(obs, act, weights1)
        loss2 = compute_loss_fn(obs, act, weights2)

        if not torch.isclose(loss1, loss2):
            print("✓ Test 4: Different weights produce different losses")
            results.append(True)
        else:
            print("✗ Test 4: Different weights should produce different losses")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 4: Error - {type(e).__name__}")
        results.append(False)

    # Test 5: Gradient flow works
    try:
        obs = torch.randn(10, 8, requires_grad=True)
        act = torch.randn(10, 2)
        weights = torch.randn(10)

        loss = compute_loss_fn(obs, act, weights)
        loss.backward()

        if obs.grad is not None:
            print("✓ Test 5: Gradient flow works")
            results.append(True)
        else:
            print("✗ Test 5: Gradients not computed")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 5: Error - {type(e).__name__}")
        results.append(False)

    return sum(results), len(results)

## Run Tests

In [77]:
# Run the tests for compute_loss
passed, total = test_compute_loss(compute_loss)
print(f"\n{'='*50}")
print(f"Results: {passed}/{total} tests passed")
if passed == total:
  print("🎉 All tests passed! You're ready to move on.")
else:
  print("⚠️  Some tests failed. Review your implementation.")

✓ Test 1: Output is scalar
✓ Test 2: Output dtype is correct
✓ Test 3: Works with different batch sizes
✓ Test 4: Different weights produce different losses
✓ Test 5: Gradient flow works

Results: 5/5 tests passed
🎉 All tests passed! You're ready to move on.


# EXERCISE 3: Implement get_policy


## Setup for Exercise 3

In [78]:
obs_dim = 8   # LunarLander observation dimension
n_acts = 2    # LunarLander action dimension
hidden_sizes = [64, 64]

# Create the policy network
policy_net = mlp([obs_dim] + hidden_sizes + [2*n_acts])

print(f"Policy network created with architecture: {obs_dim} -> {hidden_sizes} -> {2*n_acts}")

Policy network created with architecture: 8 -> [64, 64] -> 4


## TODO: Implement get_policy

In [79]:
def get_policy(obs: torch.Tensor) -> Normal:
    """
    Get the stochastic policy for a given observation (or batch of observations).

    The policy network outputs 2*n_acts values: mean and log_std for each action dimension.
    You need to:
    1. Handle single observations (add batch dimension if needed)
    2. Pass observations through the policy network
    3. Split the output into mean and log_std
    4. Clamp log_std to [-20, 2] for numerical stability
    5. Return a Normal distribution with mean and exp(log_std)

    Args:
        obs: Observation tensor of shape (obs_dim,) or (batch_size, obs_dim)

    Returns:
        Normal distribution for actions with shape (batch_size, n_acts)
    """
    # NOTE: this ensures obs is a tuple of 2 elems
    obs = obs.unsqueeze(0) if obs.dim() == 1 else obs
    batch_size = obs.shape[0]

    # NOTE: Pass obs through the policy network, to do this we need to pass a tensor through the network
    logits = policy_net(obs)

    # NOTE:Split output into mean and log_std
    mean = logits[:,:n_acts]
    log_stds = logits[:,n_acts:]

    # NOTE: clamp for numerical stability
    log_stds = torch.clamp(log_stds, min=-20, max = 2)

    return Normal(mean, torch.exp(log_stds))

## Test Functions (Feel free to collapse this cell)

In [80]:
def test_get_policy(get_policy_fn):
    """
    Test the get_policy implementation.
    """
    results = []

    # Test 1: Returns a Normal distribution
    try:
        obs = torch.randn(8)
        dist = get_policy_fn(obs)

        if isinstance(dist, Normal):
            print("✓ Test 1: Returns a Normal distribution")
            results.append(True)
        else:
            print(f"✗ Test 1: Should return Normal distribution, got {type(dist)}")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 1: Error - {type(e).__name__}")
        results.append(False)

    # Test 2: Mean has correct shape (single observation)
    try:
        obs = torch.randn(8)
        dist = get_policy_fn(obs)

        if dist.mean.shape == torch.Size([1, 2]):
            print("✓ Test 2: Mean has correct shape for single observation")
            results.append(True)
        else:
            print(f"✗ Test 2: Mean shape should be (1, 2), got {dist.mean.shape}")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 2: Error - {type(e).__name__}")
        results.append(False)

    # Test 3: Std has correct shape (single observation)
    try:
        obs = torch.randn(8)
        dist = get_policy_fn(obs)

        if dist.stddev.shape == torch.Size([1, 2]):
            print("✓ Test 3: Std has correct shape for single observation")
            results.append(True)
        else:
            print(f"✗ Test 3: Std shape should be (1, 2), got {dist.stddev.shape}")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 3: Error - {type(e).__name__}")
        results.append(False)

    # Test 4: Works with batch of observations
    try:
        obs_batch = torch.randn(10, 8)
        dist = get_policy_fn(obs_batch)

        if dist.mean.shape == torch.Size([10, 2]) and dist.stddev.shape == torch.Size([10, 2]):
            print("✓ Test 4: Works with batch of observations")
            results.append(True)
        else:
            print(f"✗ Test 4: Shapes should be (10, 2), got mean: {dist.mean.shape}, std: {dist.stddev.shape}")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 4: Error - {type(e).__name__}")
        results.append(False)

    # Test 5: Std values are positive
    try:
        obs = torch.randn(8)
        dist = get_policy_fn(obs)

        if torch.all(dist.stddev > 0):
            print("✓ Test 5: Std values are positive")
            results.append(True)
        else:
            print("✗ Test 5: Std values should be positive")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 5: Error - {type(e).__name__}")
        results.append(False)

    # Test 6: Std values are reasonable (clamping works)
    try:
        # Create observation that might produce extreme log_std values
        obs = torch.randn(8) * 100
        dist = get_policy_fn(obs)

        # Check if std is within reasonable bounds (exp(-20) to exp(2))
        min_std = torch.exp(torch.tensor(-20.0))
        max_std = torch.exp(torch.tensor(2.0))

        if torch.all(dist.stddev >= min_std - 0.01) and torch.all(dist.stddev <= max_std + 0.01):
            print("✓ Test 6: Std clamping works correctly")
            results.append(True)
        else:
            print("✗ Test 6: Std clamping might not be working")
            results.append(False)
    except Exception as e:
        print(f"✗ Test 6: Error - {type(e).__name__}")
        results.append(False)

    return sum(results), len(results)

## Run Tests

In [81]:
# Run the tests for get_policy
passed, total = test_get_policy(get_policy)
print(f"\n{'='*50}")
print(f"Results: {passed}/{total} tests passed")
if passed == total:
  print("🎉 All tests passed! You're ready to move on.")
else:
  print("⚠️  Some tests failed. Review your implementation.")

✓ Test 1: Returns a Normal distribution
✓ Test 2: Mean has correct shape for single observation
✓ Test 3: Std has correct shape for single observation
✓ Test 4: Works with batch of observations
✓ Test 5: Std values are positive
✓ Test 6: Std clamping works correctly

Results: 6/6 tests passed
🎉 All tests passed! You're ready to move on.
